In [1]:
import requests
import pandas as pd

In [2]:
# http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=6f2d4be584aa413f9df230512210306&q=45.53,87.38&date=2008-09-08&tp=1&format=json
# above is example

In [3]:
#import cleaned NFL schedule
schedule_csv = "NFL_Schedule.csv"

schedule = pd.read_csv(schedule_csv)
schedule.head()

,Full Date,New Time,Week,City,Home Team,Winner
0,2008-09-04,7,1,New York,Giants,Giants
1,2008-09-07,1,1,Philadelphia,Eagles,Eagles
2,2008-09-07,1,1,Miami,Dolphins,Jets
3,2008-09-07,1,1,New England,Patriots,Patriots
4,2008-09-07,1,1,Pittsburgh,Steelers,Steelers


In [4]:
#import cleaned NFL Stadium Locations
locations_csv = "NFL_Locations.csv"

new_df = pd.read_csv(locations_csv)
new_df

,Franchise,Latitude,Longitude
0,Chicago,41.53 N,87.38 W
1,Green Bay,44.30 N,88.01 W
2,Arizona,33.27 N,112.05 W
3,Atlanta,33.45 N,84.23 W
4,Baltimore,39.17 N,76.37 W
5,Buffalo,42.54 N,78.53 W
6,Carolina,35.14 N,80.50 W
7,Cincinnati,39.06 N,84.31 W
8,Cleveland,41.30 N,81.41 W
9,Dallas,32.47 N,96.48 W


In [6]:
locations = new_df.drop([new_df.index[3],new_df.index[9],new_df.index[20],new_df.index[19],new_df.index[17],new_df.index[11],new_df.index[12]])
locations = locations.reset_index()
locations = locations[["Franchise","Latitude","Longitude"]]
locations.replace(to_replace="New York Jets", value="New York")

,Franchise,Latitude,Longitude
0,Chicago,41.53 N,87.38 W
1,Green Bay,44.30 N,88.01 W
2,Arizona,33.27 N,112.05 W
3,Baltimore,39.17 N,76.37 W
4,Buffalo,42.54 N,78.53 W
5,Carolina,35.14 N,80.50 W
6,Cincinnati,39.06 N,84.31 W
7,Cleveland,41.30 N,81.41 W
8,Denver,39.43 N,105.01 W
9,Indianapolis,39.46 N,86.09 W
